I've referred to this blog post [https://stmorse.github.io/journal/espn-fantasy-v3.html] and the python package https://github.com/cwendt94/ff-espn-api/blob/master/ff_espn_api/league.py to better understand the undocumented ESPN Fantasy Football API. 

In [ ]:
#Edit notebook width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

To start, each view has been pulled and stored locally as .json files. Let's use json.load() to import the json as a dictionary:

In [ ]:
import json

with open('./ff_data/ff_view_mTeam.json') as f:
    data_teams = json.load(f)
    
print(data_teams)

There's a lot going on so lets isolate the keys. My first approach was to recursively loop through the JSON to find all of them but it looks like the data is both full of noise and nested structures. Let's  start with a lazy evaluation of the first couple layers:

In [ ]:
#top level keys:
l0_keys = data_teams.keys()

print(l0_keys)

This seems pretty straightforward. Lets write a helper function to evaluate the dicts and keys

In [ ]:
def dict_eval(e_dict,e_key):
    #note: e_key is passed as a list of string values
    for key in e_key:
        if isinstance(e_dict[key], dict):
            print('The key: {} stores a dict with the following keys: {}\n'.format(key, e_dict[key].keys()))
        elif isinstance(e_dict[key], list):
            print('The key: {} stores the list: {}\n'.format(key, e_dict[key]))
        else:
            print('The key: {} has the value of: {}\n'.format(key, e_dict[key]))

In [ ]:
dict_eval(data_teams, l0_keys)

The data we're after looks to be stored in 'teams' and needs some additional parsing. An initial analysis showed that there may be extra attributes depending on what options a team manager has populated so lets get a complete distinct list in one go:

In [ ]:
team_keys = []

for x in data_teams['teams']:
    for team_key in x.keys():
        if team_key not in team_keys:
            team_keys.append(team_key)
            
print(team_keys)

Each element in the list data_teams['teams'] represents a team manager in the league and returns a nested dictionary. Lets use the helper function to see what this looks like. Keys are determined per dict to account to prevent key errors:

In [ ]:
for team in data_teams['teams']:
    team_keys = team.keys()
    dict_eval(team, team_keys)

#print(type(data_teams['teams'][0]))
#print(data_teams['teams'][0])


Now we're getting somewhere! Here's what we know so far:

In [ ]:
for team in data_teams['teams']:
    team_abbrev = team['abbrev']
    team_cur_proj_rank = team['currentProjectedRank']
    team_draft_proj_rank = team['draftDayProjectedRank']
    team_div_id = team['divisionId']
    #team_logo = team['logo'] TODO: error handling for invalid keys
    team_points = team['points']
    team_points_adj = team['pointsAdjusted'] #val of adjustment points: e.g. 0 is no adjustment

We also want to see what values are available for the dicts: record, transactionCounter, and valuesByStat

In [ ]:
eval_keys = ['record', 'transactionCounter', 'valuesByStat']

for team in data_teams['teams']:
    for key in eval_keys:
        sub_key = team[key].keys()
        print(key)
        dict_eval(team[key], sub_key)
    

    

It looks like we can get high level record and transactions counts. We're not so interested in this for now because we'd like data that is at a lower grain, available in the view 'mMatchup'

In [ ]:
with open('./ff_data/ff_view_mMatchup.json') as f:
    data_matchups = json.load(f)
    
print(data_matchups)

mMatchup has a lot more of the data we're looking for so let's dive in:

In [ ]:
l0_matchup_keys = data_matchups.keys()
print(l0_matchup_keys)

Some of these keys are familiar. The bulk of the data we want is going to be in 'schedule':

In [ ]:
a = data_matchups['status'] ##This has important metadata 
print(a)

In [ ]:
t = data_matchups['schedule']
print(t)

In [ ]:
dict_eval(data_matchups, l0_matchup_keys)

In [ ]:
matchup_schedule = data_matchups['schedule']

for sch in matchup_schedule:
    if isinstance(sch, dict):
        sch_keys = sch.keys()
        print(sch_keys)
    elif isinstance(sch, list):
        print(sch)
    else:
        print(sch)



In [ ]:
for sch in matchup_schedule:
    sch_keys = sch.keys()
    dict_eval(sch, sch_keys)
    
    
        

In [ ]:
for sch in matchup_schedule :
    winner = sch['winner'] #HOME/AWAY/!UNDECIDED / TODO:ties??
    if winner == 'UNDECIDED':
        next
    else:
        game_id = sch['id']
        matchup_period_id = sch['matchupPeriodId']
        away = sch['away']
        away_team = away['teamId']
        away_cumulative_score = away['cumulativeScore']
        away_speriod_score = away['pointsByScoringPeriod']
        away_total_score = away['totalPoints']
        
        home = sch['home']
        home_team = home['teamId']
        home_cumulative_score = home['cumulativeScore']
        home_speriod_score = home['pointsByScoringPeriod']
        home_total_score = home['totalPoints']
        
        
        
        

In [ ]:
matchup_teams = data_matchups['teams']

for mteam in matchup_teams:
    
    
    if isinstance(team, dict):
        mteam_keys = mteam.keys()
        print(mteam_keys)
    elif isinstance(team, list):
        print(team)
    else:
        print(team)





In [ ]:
for mteam in matchup_teams:
    roster = mteam['roster']
    print(roster.keys())

In [ ]:
for mteam in matchup_teams:
    x = mteam['roster']['entries']
    for x in x:
        print(x.keys())

In [ ]:
for mteam in matchup_teams:
    x = mteam['roster']['appliedStatTotal']
    print(x)

In [ ]:
POSITION_MAP = {
    0: 'QB',
    1: 'TQB',
    2: 'RB',
    3: 'RB/WR',
    4: 'WR',
    5: 'WR/TE',
    6: 'TE',
    7: 'OP',
    8: 'DT',
    9: 'DE',
    10: 'LB',
    11: 'DL',
    12: 'CB',
    13: 'S',
    14: 'DB',
    15: 'DP',
    16: 'D/ST',
    17: 'K',
    18: 'P',
    19: 'HC',
    20: 'BE',
    21: 'IR',
    22: '',
    23: 'RB/WR/TE',
    24: 'ER',
    25: 'Rookie',
    'QB': 0,
    'RB': 2,
    'WR': 4,
    'TE': 6,
    'D/ST': 16,
    'K': 17,
    'FLEX': 23
}

PRO_TEAM_MAP = {
    0 : 'None',
    1 : 'ATL',
    2 : 'BUF',
    3 : 'CHI',
    4 : 'CIN',
    5 : 'CLE',
    6 : 'DAL',
    7 : 'DEN',
    8 : 'DET',
    9 : 'GB',
    10: 'TEN',
    11: 'IND',
    12: 'KC',
    13: 'OAK',
    14: 'LAR',
    15: 'MIA',
    16: 'MIN',
    17: 'NE',
    18: 'NO',
    19: 'NYG',
    20: 'NYJ',
    21: 'PHI',
    22: 'ARI',
    23: 'PIT',
    24: 'LAC',
    25: 'SF',
    26: 'SEA',
    27: 'TB',
    28: 'WSH',
    29: 'CAR',
    30: 'JAX',
    33: 'BAL',
    34: 'HOU'
}

ACTIVITY_MAP = {
    178: 'ADDED',
    180: 'ADDED',
    179: 'DROPPED',
    181: 'DROPPED',
    239: 'DROPPED',
    244: 'TRADED'
}

In [ ]:
views = ['mTeam', 'mTeam', 'mRoster', 'mSettings', 'players_wl', 'mDraftDetail', 'proTeamSchedules_wl', 'mPositionalRatings']

scoring_period_id = 'scoringPeriodId'